# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных #

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

# Импорт библиотек #

In [1]:
import  pandas as pd
from pymystem3 import Mystem
from IPython.core.display import display

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
data = pd.read_csv(filepath_or_buffer='datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
1. data['days_employed']:
    - отрицательные значения/дробные значения - как интерпретировать?
2. data['education']:
    - создать справочник 'education_dict'
3. data['education_id']:
    - вынести в справочник 'education_dict'
4. data['family_status']:
    - создать справочник 'family_status_dict'
5. data['family_status_id']:
    - вынести в справочник 'family_status_dict'
6. data['gender']:
    - изменить тип на 'category'
7. data['income_type']:
    - изменить тип на 'category'
8. data['total_income']:
    - предложить варианты сегментации заемщиков
9. data['purpose']:
    - выделить основные сегменты
    - изменить тип на 'category'

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
data.duplicated().sum()

54

In [5]:
print(data.isna().sum())
data = data.fillna(0)
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**
2. произведена замена пропущенных значений на 0 для:
    - data['days_employed']
    - data['total_income']

### Замена типа данных

#### Перевод значений строковых переменных в нижний регистр ####

In [6]:
def lower_data():
    for column in data.columns:
        if data.dtypes[column] == 'object':
            data[column] = data[column].str.strip()
            data[column] = data[column].str.lower()

lower_data()

In [7]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,f,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093.050500,на покупку своего автомобиля


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
data['days_employed'] = data['days_employed'].astype('int')
data['gender'] = data['gender'].astype('category')
data['income_type'] = data['income_type'].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int64   
 1   days_employed     21525 non-null  int32   
 2   dob_years         21525 non-null  int64   
 3   education         21525 non-null  object  
 4   education_id      21525 non-null  int64   
 5   family_status     21525 non-null  object  
 6   family_status_id  21525 non-null  int64   
 7   gender            21525 non-null  category
 8   income_type       21525 non-null  category
 9   debt              21525 non-null  int64   
 10  total_income      21525 non-null  float64 
 11  purpose           21525 non-null  object  
dtypes: category(2), float64(1), int32(1), int64(5), object(3)
memory usage: 1.6+ MB


#### Лемматизация data['purpose'] ####

In [18]:
#data['purpose'].unique()
#print(data['purpose'].astype('category'))
m = Mystem()
#t = m.lemmatize()
for purpose in data['purpose'].astype('category').cat.categories:
    lemma = ' '.join(m.lemmatize(purpose))
    for _ in range(len(data)):
        if lemma in data.loc[_, 'purpose']: data.loc[_, 'purpose'] = lemma

data['purpose']
##print(data['purpose'].astype('category').cat.categories)

KeyboardInterrupt: 

In [10]:
m = Mystem()
data['purpose'] = ' '.join(m.lemmatize(data['purpose']))

AttributeError: 'Series' object has no attribute 'splitlines'

In [ ]:
# m = Mystem()
# for _ in range(len(data)):
#     lemm = ' '.join(m.lemmatize(data.loc[_, 'purpose']))
#     data.loc[_, 'purpose'] = lemm
#
# data['purpose'].unique()

**Вывод**

### Обработка дубликатов

In [ ]:
data.duplicated().sum()

In [ ]:
print(data.shape)
data = data.drop_duplicates().reset_index(drop=True)
print(data.shape)


**Вывод**

### Лемматизация

**Вывод**

### Категоризация данных

**Вывод**

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

## Шаг 4. Общий вывод

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.